In [ ]:
from pathlib import Path
import struct

In [ ]:
FILE_PATH = Path("./asd_data/1096.asd")

In [ ]:
def read_int32(file):
    return struct.unpack("i", file.read(4))[0]

In [ ]:
# Channel encoding all in little endian
# topography: 0x5054 decodes to 'TP' in ascii little endian
# error: 0x5245 decodes to 'ER' in ascii little endian
# phase: 0x4850 decodes to 'PH' in ascii little endian
# none: 0x0000 no channel.

In [ ]:
# Read file as binary
with open(FILE_PATH, "rb") as f:
    # binary_file_data = f.read()
    print(read_int32(f))
    print(f.read(2))
    print(f.read(2))

# print(len(binary_file_data))
# print(binary_file_data[:20])
# print(read_int32(binary_file_data))

# print(str(binary_file_data))